In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

In [3]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"]=GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## Predefine tools in the previous ipynb

## Custom Tool

In [94]:
from langchain.agents import tool
@tool
def get_word_length(word:str)->int:
    """Return the word length"""
    return len(word)

get_word_length.invoke("Gyanbardhan")

11

In [8]:
@tool
def multiply(a:int,b:int)->int:
    "Multiply two numbers"
    return a*b

In [9]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [10]:
multiply.invoke({'a':10,'b':20})

200

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [107]:
search=TavilySearchResults()

In [108]:
tools=[search]

In [109]:
from langchain import hub

In [15]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [16]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

agent_scratchpad: basically a structure in which agent should work


In [110]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,tools,prompt)

In [111]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [112]:
agent_executor.invoke({"input":"Explain agentic ai with langgraph"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'agentic ai with langgraph'}`


[{'title': 'A Guide to Building Agentic RAG Systems with LangGraph', 'url': 'https://www.analyticsvidhya.com/blog/2024/07/building-agentic-rag-systems-with-langgraph/', 'content': 'We will be using LangGraph for our practical implementation of our Agentic RAG system. LangGraph, built on top of LangChain, facilitates the creation of cyclical graphs essential for developing AI agents powered by LLMs. The widely-used NetworkX library inspires its interface. It enables the coordination and checkpointing of multiple chains (or actors) through cyclic computational steps. LangGraph treats Agent workflows as a cyclical Graph structure, as depicted in the following figure. [...] One can use various frameworks to build Agentic AI systems, including CrewAI, LangChain, LangGraph, AutoGen, and many more. Using these frameworks enables us to develop complex workflows with e

{'input': 'Explain agentic ai with langgraph',
 'output': 'Agentic AI refers to AI systems capable of reasoning, decision-making, and tool usage in real-world tasks, acting autonomously. LangGraph, built on top of LangChain, facilitates the creation of cyclical graphs essential for developing these AI agents powered by LLMs. It allows for the coordination and checkpointing of multiple chains (or actors) through cyclic computational steps, treating agent workflows as a cyclical Graph structure. LangGraph helps design complex agent workflows using graphs, events, and state transitions, enabling the development of complex workflows with ease. It provides tools and libraries to create, run, and optimize LLMs in a scalable and efficient manner.'}

In [20]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [21]:
loader=WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs=loader.load()
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [22]:
documents=RecursiveCharacterTextSplitter(
    chunk_size=1000,chunk_overlap=150,
).split_documents(docs)
documents

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content='LangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024'),
 Document(metadata={'source': 'https://blog.langc

In [23]:
vector=FAISS.from_documents(documents,embeddings)
retriever=vector.as_retriever()

In [24]:
retriever.invoke("What is langgraph")

[Document(id='58023066-9305-48b7-9e5f-5024fe7a7bad', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="for production; in turn, we've heavily invested in LangGraph, leading to a stable 0.1 release this past June.LangGraph features a persistence layer that enables human-in-the-loop interactions, and it excels at building complex (i.e. more than a single LLM call) applications that require highly domain-specific cognitive architecture. Most of the agents we see in production fit this description.LangGraph is fully open source, available in both Python and Javascript. It works with or without LangChain, and integrates seamlessly with LangSmith.LangGraph Studio: Visualize and interact w

In [100]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search for information about Langgraph Studio.For any question about Langgraph Studio, ypu must use this tool!",
)

In [101]:
tools=[search,retriever_tool]

In [102]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,tools,prompt)

In [103]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=False)

In [104]:
agent_executor.invoke({"input":"Hii,what is langgraph studio"})

{'input': 'Hii,what is langgraph studio',
 'output': 'LangGraph Studio is the first IDE designed specifically for agent development, offering a way to visualize, interact with, and debug complex agentic applications. It is currently available as a desktop app for Apple Silicon and requires a LangSmith account to use. It helps developers understand the structure of their agent graphs, modify agent results, and iterate on their applications.'}

In [105]:
print(agent_executor.invoke({"input":"Waht is LoC Line of Control"})['output'])

The Line of Control (LoC) is a military control line between the Indian- and Pakistani-controlled parts of the former princely state of Jammu and Kashmir. It is not a legally recognized international boundary but serves as the de facto border. It was established as part of the Simla Agreement at the end of the Indo-Pakistani War of 1971.


## ReAct(Reasoning & Action)
### How it works:-
- Agent retrieves the question 
- It reasons in natural language about how to solve it.
- Based on its reasoning, it takes action.(e.g., searching a database,calculating something)
- The results of these are integrated into its reasoning and trigger further actions.
- It repeats the process until it arrives its solution.

Combines thinking and doing (reasoning and action)
Performs iterative steps, updating its process based on action results.
Typically handles complex decision-making scenerios.

![ReAct Agent](ReAct_Agent.gif)

In [32]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool,AgentExecutor,create_react_agent
from langchain_core.prompts import PromptTemplate

In [33]:
google_search=GoogleSerperAPIWrapper()

In [34]:
tools=[
    Tool(
        name="Web Answer by search",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [35]:
template='''Answer the following question as best you can.You have access to the following tools:{tools}
Use the following format:
Question: the input question you must question
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input of the action
Observation: the result of the action
.... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought: {agent_scratchpad}
'''

In [36]:
prompt=PromptTemplate.from_template(template)

In [37]:
search_agent=create_react_agent(llm,tools,prompt)

In [38]:
agent_executor=AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [39]:
agent_executor({'input':"Where is the home town of the 2024 T20 World Cup cricket winner captain and also tell its contribution with every match and its contribution in winning"})



> Entering new AgentExecutor chain...


C:\Users\gyanb\AppData\Local\Temp\ipykernel_21672\4197397583.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor({'input':"Where is the home town of the 2024 T20 World Cup cricket winner captain and also tell its contribution with every match and its contribution in winning"})


Thought:  I need to know who won the 2024 T20 World Cup to answer this question.  
Action: Web Answer by search
Action Input: 2024 T20 World Cup winnerIndia won all their matches, and were the first team to win a T20 World Cup undefeated. They joined England and West Indies as the only teams to win the title ... Series result, India won the 2024 ICC Men's T20 World Cup ; Match number, T20I no. 2729 ; Match days, 29 June 2024 - day (20-over match) ; Umpires. New Zealand. In the second innings, South Africa managed to post a score of 169/8 thus India claimed the victory by 7 runs to win their second T20 World Cup title. Virat ... India script stunning title win | Match Highlights | SA v IND | T20WC 2024 Final ... ICC Men's T20 World Cup, 2024. Staten Island Cricket Club - ... The moment India clinched the ICC Men's T20 World Cup 2024 trophy · Comments1.8K. Winner. India. Matches. RESULT. Final, Bridgetown, June 29, 2024, ICC Men's T20 World Cup. IND Flag. IND. 176/7. SA Flag. SA. (20 ov,

{'input': 'Where is the home town of the 2024 T20 World Cup cricket winner captain and also tell its contribution with every match and its contribution in winning',
 'output': 'The home town of the captain of the 2024 T20 World Cup winning team, Rohit Sharma, is Bansod, Nagpur, Maharashtra, India. Rohit Sharma was the captain of the winning team, India, and contributed as a batsman throughout the tournament. He led his team to victory in a nail-biting final against South Africa.',
 'intermediate_steps': [(AgentAction(tool='Web Answer by search', tool_input='2024 T20 World Cup winner', log='Thought:  I need to know who won the 2024 T20 World Cup to answer this question.  \nAction: Web Answer by search\nAction Input: 2024 T20 World Cup winner'),
   "India won all their matches, and were the first team to win a T20 World Cup undefeated. They joined England and West Indies as the only teams to win the title ... Series result, India won the 2024 ICC Men's T20 World Cup ; Match number, T20I 

In [40]:
agent_executor({'input':" Who is the god of cricket what is his heighest score in odis,test,t20, and ipl.Which is the last match palyed by him for his country."})



> Entering new AgentExecutor chain...
Thought:  This question asks about a fictional "god of cricket" and real-world cricket statistics.  It's likely a playful question.

Thought: To answer the first part, I'll need to find information about a figure often called the "God of Cricket".  

Action: Web Answer by search
Action Input: Who is called the God of Cricket Fondly called the “God of Cricket” or “Master Blaster,” Sachin Tendulkar is considered one of the greatest batsmen of all time in the sport that is considered a national institution in India, cricket.Thought:  I know Sachin Tendulkar is referred to as the "God of Cricket". Now I need to find his cricket statistics.

Action: Web Answer by search
Action Input: Sachin Tendulkar ODI, Test, T20, IPL statsAllround: Batting and bowling · 1000 runs and 100 wickets · 250 runs and 10 wickets in a series · A hundred and four wickets in an innings. Allround: Batting, ... Sachin Tendulkar Career Stats ; Tests, 200, 329 ; ODIs, 463, 452 ; 

{'input': ' Who is the god of cricket what is his heighest score in odis,test,t20, and ipl.Which is the last match palyed by him for his country.',
 'output': 'Sachin Tendulkar is often called the "God of Cricket".  His highest scores in different formats are:\n\n* **Tests:** 248*\n* **ODIs:** 200*\n* **T20Is:** 10\n* **IPL:**  No specific highest score provided in the results. \n\nThe last match Sachin Tendulkar played for India was against  **West Indies on November 16, 2013**.',
 'intermediate_steps': [(AgentAction(tool='Web Answer by search', tool_input='Who is called the God of Cricket', log='Thought:  This question asks about a fictional "god of cricket" and real-world cricket statistics.  It\'s likely a playful question.\n\nThought: To answer the first part, I\'ll need to find information about a figure often called the "God of Cricket".  \n\nAction: Web Answer by search\nAction Input: Who is called the God of Cricket '),
   'Fondly called the “God of Cricket” or “Master Blaster

In [50]:
@tool
def multiplication(a,b):
    "Multiply two numbers"
    return a*b

In [51]:
@tool
def addition(a,b):
    "Add two numbers"
    return a+b

In [52]:
@tool
def substraction(a,b):
    "Substract a from b"
    return a-b

In [53]:
@tool
def division(a,b):
    "Divide two numbers"
    return a/b

In [54]:
@tool
def modulas(a,b):
    "Remainder of two numbers"
    return a%b

In [55]:
tools = [
    Tool.from_function(
        name="Addition",
        func=addition,
        description="Useful for when you need to add two numbers. Input must be two numbers."
    ),
    Tool.from_function(
        name="Subtraction",
        func=substraction,
        description="Useful for when you need to subtract two numbers. Input must be two numbers."
    ),
    Tool.from_function(
        name="Multiplication",
        func=multiplication,
        description="Useful for when you need to multiply two numbers. Input must be two numbers."
    ),
    Tool.from_function(
        name="Division",
        func=division,
        description="Useful for when you need to divide two numbers. Input must be two numbers."
    ),
    Tool.from_function(
        name="Remainder/Modulus",
        func=modulas,
        description="Useful for when you need to find remainder of two numbers. Input must be two numbers."
    ),
]

In [56]:
calculator_agent=create_react_agent(llm,tools,prompt)

In [57]:
agent_executor=AgentExecutor(agent=calculator_agent,tools=tools,verbose=True)

In [58]:
agent_executor.invoke({'input':'Multiply a= 5 and b=2'})



> Entering new AgentExecutor chain...
Thought: I need to use the Multiplication tool.
Action: Multiplication
Action Input: 5 2

ValidationError: 1 validation error for multiplication
b
  Field required [type=missing, input_value={'a': '5 2'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [59]:
agent_executor.invoke({'input':'Value of 2+5*36/6-7+2'})



> Entering new AgentExecutor chain...
Thought:  We need to follow the order of operations (PEMDAS/BODMAS) to solve this: Multiplication and Division first, then Addition and Subtraction.

Action: Multiplication
Action Input: 5 * 36

ValidationError: 1 validation error for multiplication
b
  Field required [type=missing, input_value={'a': '5 * 36'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [61]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

# Tool: A simple calculator function
def calculator_tool(query: str) -> str:
    try:
        result = eval(query)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Wrap it in LangChain's Tool interface
calc_tool = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Useful for when you need to do math calculations. Input should be a valid Python expression like '2 + 2' or '3 * (4 + 1)'."
)



# Initialize the agent with the tool
agent = initialize_agent(
    tools=[calc_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Test the agent
response = agent.run("What is  2+5*36/6-7+2?")
print("Answer:", response)




> Entering new AgentExecutor chain...
Thought: I need to follow the order of operations (PEMDAS/BODMAS) to solve this.
Action: Calculator
Action Input: 2 + 5 * 36 / 6 - 7 + 2
Observation: 27.0
Thought:Thought: I now know the final answer
Final Answer: 27.0 


> Finished chain.
Answer: 27.0


In [63]:
@tool
def get_emp_id(name):
    """
    To get the employee id, it takes employee name as arguments 
    name(str): Name of the the employee
    """
    employee={
        "Gyanbardhan":"E001",
        "Lakshit Upreti":"E002",
        "Daulat Ojha":"E003",
        "Yash Gangan":"E004",
        "Guransh Singh":"E005",
        "Aditya Prasad":"E006",
        "Harsh Verma":"E007",
        "Vasu Kapasia":"E008"
    }
    return employee.get(name,"Employee not found")

In [65]:
@tool
def get_emp_sal(emp_id):
    """To get the salary of employee, it tales emp_id as input and return salary"""
    salaries={
        "E001":"18 LPA",
        "E002":"12 LPA",
        "E003":"21.5 LPA",
        "E004":"12 LPA",
        "E005":"14 LPA",
        "E006":"13 LPA",
        "E008":"21.5 LPA"
    }
    return salaries.get(emp_id,"Employee not found")

In [68]:
prompt=hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [69]:
tools=[get_emp_id,get_emp_sal]

In [72]:
emp_agent=create_react_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=emp_agent,tools=tools,verbose=True)

In [77]:
agent_executor.invoke({'input':'Salary of  Gyanbardhan'})



> Entering new AgentExecutor chain...
Thought: I need to first find Gyanbardhan's employee ID using his name. Then, I can use his employee ID to find his salary. 
Action: get_emp_id
Action Input: GyanbardhanE001Thought: Now that I have Gyanbardhan's employee ID (E001), I can use it to find his salary.
Action: get_emp_sal
Action Input: E00118 LPAI now know the final answer
Final Answer:  18 LPA 


> Finished chain.


{'input': 'Salary of  Gyanbardhan', 'output': '18 LPA'}

In [79]:
agent_executor.invoke({'input':'What is the Salary of  Lakshit Upreti'})



> Entering new AgentExecutor chain...
Thought: To find Lakshit Upreti's salary, I need to first find their employee ID using their name. 
Action: get_emp_id
Action Input: Lakshit UpretiE002Thought: Now that I have the employee ID, I can use it to find Lakshit Upreti's salary.
Action: get_emp_sal
Action Input: E00212 LPA Thought: I now know the final answer
Final Answer: 12 LPA 


> Finished chain.


{'input': 'What is the Salary of  Lakshit Upreti', 'output': '12 LPA'}

## 🧠 Self-Ask Agent


- **Self-Ask** is an agent architecture where an LLM decomposes complex questions into sub-questions.
- Introduced in the paper **“Large Language Models are Zero-Shot Reasoners”**.
- Enables **multi-step reasoning** and the use of **external tools** like calculators or search APIs.


#### 🔄 Working Mechanism

1. Receive Query
- Input: A complex or multi-step question.
  - Example: `"What is 25 squared minus 100?"`

2. Decompose Into Sub-Questions
- Breaks the main question into simpler ones:
  - "What is 25 squared?"
  - "Then subtract 100."

<img src="Self_Ask.png" alt="Self Ask" style="width:300px; height:auto;"/>

3. Use External Tools
- Uses tools (like a calculator) to answer intermediate questions.

4. Combine Results
- Final answer is derived from the sub-results.

![Self Rag](Self_Ask_Agent.png)


In [81]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [82]:
from langchain import hub
from langchain.agents import AgentExecutor,create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [87]:
tools=[
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True,
    )
]

In [88]:
prompt=hub.pull("hwchase17/self-ask-with-search")

In [89]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [91]:
agent_self_ask=create_self_ask_with_search_agent(llm,tools,prompt)

In [92]:
agent_executor=AgentExecutor(agent=agent_self_ask,tools=tools,verbose=True,handle_parsing_errors=True)

In [93]:
agent_executor.invoke({'input':'What is the home town of current Prime Minister of India'})



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is the current Prime Minister of India?Narendra ModiFollow up: What is Narendra Modi's home town?Modi was born and raised in Vadnagar, Gujarat, where he completed his secondary education. He was introduced to the RSS at the age of eight. Modi became a full-time worker for the RSS in Gujarat in 1971. The RSS assigned him to the BJP in 1985 and he rose through the party hierarchy, becoming general secretary in 1998.So the final answer is: Vadnagar, Gujarat

> Finished chain.


{'input': 'What is the home town of current Prime Minister of India',
 'output': 'Vadnagar, Gujarat'}